In [1]:
from src.datasets.data_module import CloudCoverDataModule
from pathlib import Path
from src.models.deeplab_unet import LightningDeeplab
from src.training.trainer import train
from src.testing.tester import test

### Dataloader

In [2]:
data_module = CloudCoverDataModule(
    train_X_folder_path=Path("../data/final/public/train_features/"),
    train_y_folder_path=Path("../data/final/public/train_labels/"),
    test_X_folder_path=Path("../data/final/private/test_features/"),
    test_y_folder_path=Path("../data/final/private/test_labels/"),
    train_batch_size=4,
    val_batch_size=4,
    test_batch_size=4,
    val_size=0.2,
    random_state=42
)

In [3]:
data_module.prepare_data()

In [4]:
data_module.setup(stage="fit")
data_module.setup(stage="test")

### Train

In [5]:
# Si disponible, prendre la version la plus à jour du modèle Unet entraîné
unet_model = "../code/logs/unet/version_0/checkpoints/unet-epoch=21-val_loss=0.23.ckpt"

deeplab = train(
    model=LightningDeeplab(4, 2, unet_model, bilinear=True, learning_rate=0.001),
    run_name="deeplab_unet",
    model_version=0,
    data_module=data_module,
    max_epochs=2,
    patience=5
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params
------------------------------------------------------
0 | model          | DeepLabV3          | 37.0 M
1 | train_jaccard  | BinaryJaccardIndex | 0     
2 | train_accuracy | BinaryAccuracy     | 0     
3 | val_jaccard    | BinaryJaccardIndex | 0     
4 | val_accuracy   | BinaryAccuracy     | 0     
5 | test_jaccard   | BinaryJaccardIndex | 0     
6 | test_accuracy  | BinaryAccuracy     | 0     
------------------------------------------------------
37.0 M    Trainable params
0         Non-trainable params
37.0 M    Total params
148.010   Total estimated model params size (MB)


Sanity Checking: |                                                                               | 0/? [00:00<…

C:\Users\ultav\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


torch.Size([4, 2, 512, 512])


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.00 GiB. GPU 0 has a total capacty of 6.00 GiB of which 0 bytes is free. Of the allocated memory 12.16 GiB is allocated by PyTorch, and 27.89 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### Test  
Replace the checkpoint path with the best checkpoint from the training.

In [ ]:
#deeplab = LightningDeeplab.load_from_checkpoint(checkpoint_path='./logs/deeplab/version_0/checkpoints/deeplab-epoch=01-val_loss=0.67.ckpt', n_channels=4, n_classes=2)

In [ ]:
test(
    model=deeplab,
    run_name="deeplab",
    model_version=0,
    data_module=data_module
)